In [57]:
#Yunwei Zhang, Xiaoyue Zhu,Ziqing Yuan
import pandas as pd

data = pd.read_csv("CollegeScorecard.csv")


C:\Users\derek\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7,9,1427,1542,1561,1575,1725,1726,1727,1728) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Safety

In [58]:
crime = pd.read_csv("Crime_2015.csv", thousands=",", index_col=11)[
    [
        "ViolentCrime",
        "Murder",
        "Rape",
        "Robbery",
        "AggravatedAssault",
        "PropertyCrime",
        "Burglary",
        "Theft",
        "MotorVehicleTheft",
    ]
].dropna() # Select the features related to the crime
crime.describe() 


,ViolentCrime,Murder,Rape,Robbery,AggravatedAssault,PropertyCrime,Burglary,Theft,MotorVehicleTheft
count,371.000000,371.000000,371.000000,371.000000,371.000000,371.000000,371.000000,371.000000,371.000000
mean,364.630997,4.601887,40.936388,77.601078,241.493801,2623.247439,539.000539,1894.849057,189.396496
std,181.063649,3.398405,19.439597,53.605059,137.283766,821.408694,229.047572,571.432434,133.020604
min,61.600000,0.000000,2.700000,2.300000,25.500000,420.000000,159.700000,226.500000,15.700000
25%,236.400000,2.200000,28.650000,36.750000,144.400000,2021.700000,365.500000,1486.250000,96.700000
50%,331.800000,3.900000,37.400000,67.100000,215.700000,2588.600000,503.600000,1854.000000,154.300000
75%,447.550000,6.350000,50.000000,105.500000,305.500000,3208.100000,673.550000,2257.700000,245.850000
max,1160.000000,20.600000,165.600000,351.900000,981.300000,5190.600000,1503.700000,3936.400000,718.600000


In [59]:
crime = crime.apply(lambda x: (x - x.min()) / (x.max() - x.min())) # Use normalization to make the data for each feature between 0 and 1
crime

,ViolentCrime,Murder,Rape,Robbery,AggravatedAssault,PropertyCrime,Burglary,Theft,MotorVehicleTheft
City,,,,,,,,,
Abilene,0.319465,0.257282,0.327195,0.217677,0.258736,0.668469,0.515104,0.611095,0.352397
Akron,0.160961,0.247573,0.217925,0.208524,0.098661,0.446988,0.309226,0.438422,0.154218
Albany,0.551985,0.378641,0.170043,0.445080,0.466939,0.728231,0.699330,0.654007,0.179257
Albany,0.047979,0.121359,0.156538,0.052632,0.039234,0.584497,0.241741,0.606378,0.330061
Albuquerque,0.665513,0.296117,0.375077,0.584668,0.513183,0.877835,0.538467,0.760425,0.940674
...,...,...,...,...,...,...,...,...,...
Guayama,0.172979,0.553398,0.022099,0.206808,0.139987,0.084518,0.078720,0.082105,0.015507
Mayaguez,0.160142,0.558252,0.015347,0.228833,0.118330,0.188656,0.161756,0.171191,0.093328
Ponce,0.154588,0.873786,0.014119,0.182780,0.122097,0.097472,0.040699,0.109410,0.031868


In [60]:
crime_city = crime.sum(axis=1) #  Add all the features with a weight of 1 and come up with a final crime point
crime_city = (
    (crime_city - crime_city.min()) / (crime_city.max() - crime_city.min()) * -1
) # normalize the data again
crime_city


City
Abilene       -0.555550
Akron         -0.334197
Albany        -0.688209
Albany        -0.316047
Albuquerque   -0.915522
                 ...   
Guayama       -0.169483
Mayaguez      -0.229886
Ponce         -0.217615
San German    -0.000000
San Juan      -0.344992
Length: 371, dtype: float64

In [73]:
crime_city = crime_city.to_dict() 
crime_ = data.loc[:, ["INSTNM", "CITY"]]
crime_score = crime_.CITY.map(lambda x: crime_city.get(x))  # for every candidate city,search its crime points
crime_score


AttributeError: 'dict' object has no attribute 'to_dict'

In [ ]:
crime_score = crime_score.fillna(crime_score.mean()) # To deal with the nan value, some candidate cities do not have a corresponding crime profile score, which we treat as the average crime profile score level

crime_score


# Urban
Only cities with feature values of 11 12 13 need to be scored under the feature LOCALE and all other cities are scored -10

In [74]:
urban = data.loc[:, ["INSTNM", "LOCALE"]] 
urban.describe()


,LOCALE
count,7380.000000
mean,19.589024
std,9.380431
min,11.000000
25%,12.000000
50%,21.000000
75%,22.000000
max,43.000000


In [75]:
rule = {
    11: 10,
    12: 9,
    13: 8,
} # Rate the candidate cities out of 10, in descending order
score = urban["LOCALE"].map(lambda x: rule.get(x)).fillna(-10) 
maximum = score.max()
minimum = score.min()
urban_score = score.map(lambda x: (x - minimum) / (maximum - minimum)) # normalization
urban_score


0       0.95
1       0.95
2       0.95
3       0.95
4       0.95
        ... 
7799    0.00
7800    0.00
7801    0.00
7802    0.00
7803    0.00
Name: LOCALE, Length: 7804, dtype: float64

# Diversity
Diversity consists of racial diversity and major diversity

In [76]:
diversity = data.loc[
    :,
    [
        "INSTNM",
        "HBCU",
        "PBI",
        "ANNHI",
        "TRIBAL",
        "AANAPII",
        "HSI",
        "NANTI",
        "MENONLY",
        "WOMENONLY",
        "CIP01BACHL",
        "CIP03BACHL",
        "CIP04BACHL",
        "CIP05BACHL",
        "CIP09BACHL",
        "CIP10BACHL",
        "CIP11BACHL",
        "CIP12BACHL",
        "CIP13BACHL",
        "CIP14BACHL",
        "CIP15BACHL",
        "CIP16BACHL",
        "CIP19BACHL",
        "CIP22BACHL",
        "CIP23BACHL",
        "CIP24BACHL",
        "CIP25BACHL",
        "CIP26BACHL",
        "CIP27BACHL",
        "CIP29BACHL",
        "CIP30BACHL",
        "CIP31BACHL",
        "CIP38BACHL",
        "CIP39BACHL",
        "CIP40BACHL",
        "CIP41BACHL",
        "CIP42BACHL",
        "CIP43BACHL",
        "CIP44BACHL",
        "CIP45BACHL",
        "CIP46BACHL",
        "CIP47BACHL",
        "CIP48BACHL",
        "CIP49BACHL",
        "CIP50BACHL",
        "CIP51BACHL",
        "CIP52BACHL",
        "CIP54BACHL",
    ],
].fillna(0)
diversity.describe()


,HBCU,PBI,ANNHI,TRIBAL,AANAPII,HSI,NANTI,MENONLY,WOMENONLY,CIP01BACHL,...,CIP44BACHL,CIP45BACHL,CIP46BACHL,CIP47BACHL,CIP48BACHL,CIP49BACHL,CIP50BACHL,CIP51BACHL,CIP52BACHL,CIP54BACHL
count,7804.000000,7804.000000,7804.000000,7804.000000,7804.000000,7804.000000,7804.000000,7804.000000,7804.000000,7804.000000,...,7804.000000,7804.000000,7804.000000,7804.000000,7804.000000,7804.000000,7804.000000,7804.000000,7804.000000,7804.000000
mean,0.012686,0.004485,0.001281,0.004357,0.001794,0.016914,0.001666,0.008585,0.005510,0.050999,...,0.208611,0.258329,0.016914,0.026525,0.014352,0.026653,0.303306,0.371348,0.427089,0.238596
std,0.111922,0.066823,0.035776,0.065866,0.042320,0.128959,0.040783,0.092264,0.074029,0.295140,...,0.557403,0.592923,0.167823,0.207975,0.156205,0.213733,0.614561,0.688775,0.712165,0.572528
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


# Diversity

Give negative points to these schools with low racial diversity

In [77]:
diversity.iloc[:, 1:10] = (
    diversity.iloc[:, 1:10] * -1 * 38
)  # racial weight is 38 because major diversity has 38


# Major diversity

It is assumed that the more disciplines the school has, the higher the diversity score
In this case, because the client is studying education, a score of -10 is recorded for schools without a bachelor's degree in education

In [78]:
diversity.iloc[:, 10:] = (diversity.iloc[:, 10:] - 1) * 9
diversity[diversity["CIP13BACHL"] == 0].loc[
    :, "CIP13BACHL"
] = -10  # if the institute does not have bachelor for education, set it to -10


C:\Users\derek\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [79]:
score = diversity.apply(lambda x: x[1:].sum(), axis=1) # calculate the sum
maximum = score.max()
minimum = score.min()
diversity_score = score.map(lambda x: (x - minimum) / (maximum - minimum)) # normalization
diversity_score


0       0.620896
1       0.704478
2       0.220896
3       0.597015
4       0.486567
          ...   
7799    0.113433
7800    0.113433
7801    0.113433
7802    0.113433
7803    0.113433
Length: 7804, dtype: float64

# Quality
Among schools with an admission rate below 0.7, select schools with higher values of average SAT and highest degree awarded

In [80]:
quality = data.loc[:, ["INSTNM", "SAT_AVG", "HIGHDEG", "ADM_RATE"]]
quality.describe()


,SAT_AVG,HIGHDEG,ADM_RATE
count,1420.000000,7804.00000,2220.000000
mean,1056.689437,2.17632,0.682432
std,131.816683,1.31764,0.209073
min,666.000000,0.00000,0.000000
25%,971.750000,1.00000,0.552275
50%,1036.500000,2.00000,0.699650
75%,1117.250000,4.00000,0.834125
max,1534.000000,4.00000,1.000000


In [81]:
quality.fillna(0)
quality["SAT_AVG"] = (quality["SAT_AVG"] - quality["SAT_AVG"].min()) / (
    quality["SAT_AVG"].max() - quality["SAT_AVG"].min()
)
quality["HIGHDEG"] = (quality["HIGHDEG"] - quality["HIGHDEG"].min()) / (
    quality["HIGHDEG"].max() - quality["HIGHDEG"].min()
)
quality["ADM_RATE"] = quality["ADM_RATE"].map(lambda x: 1 if x < 0.7 else 0) # The median of admission rate is 0.699650, so we choose the schools whose admission rate that is lower than the median
score = quality.apply(lambda x: x[1:].sum(), axis=1)
quality_score = (score - score.min()) / (score.max() - score.min())
quality_score


0       0.393625
1       0.517665
2       0.333333
3       0.530722
4       0.729647
          ...   
7799    0.000000
7800    0.000000
7801    0.000000
7802    0.000000
7803    0.000000
Length: 7804, dtype: float64

# Integrate


In [82]:
integration = pd.concat(
    [urban_score, diversity_score, quality_score, crime_score], axis=1
)
integration.index = data.INSTNM
integration.columns = ["urban_score", "diversity_score", "quality_score", "crime_score"]
integration["SCORE"] = integration.sum(axis=1)
integration.sort_values(by="SCORE", inplace=True, ascending=False)
integration


,urban_score,diversity_score,quality_score,crime_score,SCORE
INSTNM,,,,,
Brigham Young University-Provo,0.95,0.865672,0.901306,-0.076860,2.640117
University of Minnesota-Twin Cities,1.00,0.919403,0.898618,-0.314883,2.503137
University of Florida,0.95,0.838806,0.899386,-0.195159,2.493033
University of Wisconsin-Madison,0.95,0.811940,0.897849,-0.202770,2.457020
Portland State University,1.00,0.785075,0.812212,-0.141101,2.456186
...,...,...,...,...,...
Vista College,0.00,0.113433,0.000000,-0.840314,-0.726881
Strayer University-Alexandria Campus,0.00,0.113433,0.000000,-0.844671,-0.731238
Strayer University-Shelby Oaks Campus,0.00,0.113433,0.000000,-0.927847,-0.814415


In [83]:
list(integration.head(40).index)

['Brigham Young University-Provo',
 'University of Minnesota-Twin Cities',
 'University of Florida',
 'University of Wisconsin-Madison',
 'Portland State University',
 'University of Virginia-Main Campus',
 'George Washington University',
 'University of Michigan-Ann Arbor',
 'Syracuse University',
 'New York University',
 'Utah State University',
 'Northeastern University',
 'Ohio State University-Main Campus',
 'Liberty University',
 'University of Nebraska-Lincoln',
 'Saint Louis University',
 'North Carolina State University at Raleigh',
 'University of North Florida',
 'Texas A & M University-College Station',
 'University of Louisiana at Lafayette',
 'Augsburg College',
 'Virginia Commonwealth University',
 'University of Kentucky',
 'San Jose State University',
 'Michigan State University',
 "St John's University-New York",
 'University of Southern Mississippi',
 'DePaul University',
 'University of North Texas',
 'University of Illinois at Urbana-Champaign',
 'Yale University',